# Detecting terrestrial heatwaves in Australia  

The purpose of this notebook is to take in daily tmax data, along with previously computed mean climatology and threshold values, process these and output all the THW events that occur in the dataset

First, we import the data and required modules

# Importing packages and defining functions

In [1]:
from dask.distributed import LocalCluster, Client
import dask.array
import datetime
from datetime import date 
from datetime import datetime
import glob
import numpy as np
import scipy.ndimage as ndimage
from scipy.ndimage.measurements import label, find_objects
import xarray as xr

In [2]:
# This is for running on Gadi

import os
import dask.distributed
threads_per_worker = 1
try:
    c # Already running
except NameError:
    c = dask.distributed.Client(
        n_workers=int(os.environ['PBS_NCPUS'])//threads_per_worker,
        threads_per_worker=threads_per_worker,
        memory_limit=f'{4*threads_per_worker}gb',
        local_directory=os.path.join(os.environ['PBS_JOBFS'],
                                     'dask-worker-space')
    )
c

KeyError: 'PBS_NCPUS'

This function will make the clim and thresh repeatable so that the shapes of clim, thresh and obs are equal.

In [3]:
def fix(ds):
    trial = ds
    i = 0
    while i < 37:  # <-- Here, 37 corresponds to the number of years in the observation dataset 
        trial = xr.concat([trial, ds], 'dayofyear')
        i+=1
    trial = trial.isel(dayofyear = slice(0,13514))
    # to specify the dates/time 
    trial.coords['dayofyear'] = np.arange(date(1982,1,1).toordinal(),date(2018,12,31).toordinal()+1) 
    
    # This code was used to rename the dayofyear dimension to time.
    trial['time'] = trial['dayofyear']
    del trial['dayofyear']
    trial = trial.rename({'dayofyear': 'time'})
    
    t = np.arange(date(1982,1,1).toordinal(),date(2018,12,31).toordinal()+1)
    dates = [date.fromordinal(tt.astype(int)) for tt in t]
    
    trial.coords['time'] = dates

    return trial

The function 'calc_severity()' calculates severity based on the severity index introduced by Hobday et al (2018) for categorizing marine heatwaves.

This function merges together the chunks along the time dimension by setting the 'time' chunk size to None, so that the resulting chunks each contain the full time series for that location. To run the severity calculaion on each timeseries chunk, '.map_blocks()' is used so that the calculation can be done in parallel.

In [4]:
def calc_severity(data, new_climatology, new_threshold):

    threshold_anomaly = new_threshold - new_climatology
    
    # Ignore divide by zero errors
    nperr = np.seterr(divide='ignore')

    def calc_severity_helper(da, *args, **kwargs):
        if da.size == 0:
            return da
    
        coords = {}
        for k,v in obs_aus_tmax.coords.items():
            if k == 'time':
                continue
            coords[k] = slice(v[0], v[-1])
    
        anomaly = da - new_climatology.sel(coords)
        severity = anomaly / threshold_anomaly.sel(coords)
        return severity

    data = data.chunk({'time': None})
    r = data.map_blocks(calc_severity_helper)
    
    np.seterr(**nperr)
    
    return r

This function returns values with at least n contiguous points around them. I am using n=3, to define a terrestrial heatwave as an event where the daily tmax exceeds the 90th percentile for at least 3 consecutive days (Perkins and Alexander, 2013)

In [6]:
def atleastn(da, n, dim='time'):

    def atleastn_helper(array, n, axis):
        count = np.zeros_like(np.take(array, 0,axis=axis), dtype='i4')
        mask = np.empty_like(np.take(array, 0,axis=axis), dtype='bool')
        mask = True
    
        for i in range(array.shape[axis]):
            array_slice = np.take(array, i, axis=axis)
        
            # Increase the count when there is a valid value, reset when there is not
            # This was initially set to 0, now I have changed it to 1 to detect only valid heatwave days 
            # The previous way was fine as long as I masked values less than or equal to 1, and they were white on the colour bar
            count = np.where(array_slice > 1, count + 1, 0)
        
            # Add new points when the contiguous count exceeds the threshold
            mask = np.where(count >= n, False, mask)
            
        out_slice = np.take(array, array.shape[axis]//2, axis=axis)
        return np.where(mask, np.nan, out_slice)
    
    def atleastn_dask_helper(array, axis, **kwargs):
        r = dask.array.map_blocks(atleastn_helper, array, drop_axis=axis, axis=axis, n=n, dtype=array.dtype)
        return r
    
    if isinstance(da.data, dask.array.Array):
        reducer = atleastn_dask_helper
    else:
        reducer = atleastn_helper
        
    return da.rolling({dim: n*2-1}, center=True, min_periods=n).reduce(reducer, n=n)


## Opening files and loading data

In [7]:
mean_climatology = xr.open_dataarray('/g/data/e14/cp3790/Charuni/climatology-australia.nc')
threshold = xr.open_dataarray('/g/data/e14/cp3790/Charuni/threshold-australia.nc')

In [8]:
files = sorted(glob.glob('/g/data/e14/cp3790/Charuni/ERA5-new/era5_dailytmax_*.nc'))

obs_aus = (xr.open_mfdataset(files, combine='nested', concat_dim='time', chunks={'latitude': 10})
           .sel(time=slice('1982', '2018'), longitude=slice(113, 154), latitude=slice(-10, -44)))
obs_aus_tmax = obs_aus["dmax"]
obs_aus_tmax.attrs['units'] = 'deg C'

Making the mean climatology and 90th percentile climatology (threshold) repeatable so that their sizes are compatible with the size of the daily tmax data using the 'fix' function

In [9]:
new_climatology = fix(mean_climatology)

In [10]:
new_threshold = fix(threshold)

## Calculations 

Calculating severity using 'map_blocks()', which enables the severity calculation to run on each timeseries chunk, facilitating parallel computation 

In [11]:
thw_severity_chai = calc_severity(obs_aus_tmax, new_climatology, new_threshold)

We are only interested in the days where the daily tmax has exceeded the threshold. If severity > 1, this means that daily tmax > threshold (from the definition of severity). 

In [ ]:
%%time
%matplotlib inline

candidates = thw_severity_chai.where(thw_severity_chai > 1)

Next, we mask out points where there are less than 3 contiguous points in the time dimension

In [13]:
%%time

atleastn(candidates, n=3)

CPU times: user 53 ms, sys: 3 ms, total: 56 ms
Wall time: 54.1 ms


<xarray.DataArray (time: 13514, latitude: 137, longitude: 165)>
dask.array<where, shape=(13514, 137, 165), dtype=float32, chunksize=(13514, 10, 165), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float64 -10.0 -10.25 -10.5 ... -43.5 -43.75 -44.0
  * time       (time) datetime64[ns] 1982-01-01 1982-01-02 ... 2018-12-31
  * longitude  (longitude) float64 113.0 113.2 113.5 113.8 ... 153.5 153.8 154.0

## Test on 1D array 

In [14]:
test = xr.DataArray([3, 1, 4, 0, 0, 1, 4, 0, 1, 1, 3, 0, 2, 4, 0, 1, 2, 4, 2, 0], dims=['time'])

atleastn(test, 3)

<xarray.DataArray (time: 20)>
array([ 3.,  1.,  4., nan, nan, nan, nan, nan,  1.,  1.,  3., nan, nan,
       nan, nan,  1.,  2.,  4.,  2., nan])
Dimensions without coordinates: time

## Saving to netCDF

All the terrestrial heatwave days across Australia from 1982-2018 will be stored in filtered_severity_final_new_mask.nc 

In [14]:
%%time

oscar = atleastn(candidates, n=3)

xr.Dataset({'severity': oscar}).to_netcdf('/g/data/e14/cp3790/Charuni/filtered_severity_final_new_mask.nc',
                                              encoding={'severity': 
                                                        {'chunksizes': (100, oscar.shape[1], oscar.shape[2]),
                                                         'zlib': True,
                                                         'shuffle': True, 
                                                         'complevel': 2}})   

# compression level (complevel) up to 6 is fine, >6 and it starts giving trouble 

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.10/lib/python3.6/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in greater


CPU times: user 2min 40s, sys: 8min 5s, total: 10min 45s
Wall time: 11min 58s
